In [ ]:
import numpy as np
import pandas as pd

# pd.set_option('display.height', 1000)
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.width", 1000)
pd.set_option("max_colwidth", 1000)
from IPython.core.display import HTML, display

display(HTML("<style>.container { width:100% !important; }</style>"))


def df_stats(df):
    from tabulate import tabulate

    print("\n***** Shape: ", df.shape, " *****\n")

    columns_list = df.columns.values.tolist()
    isnull_list = df.isnull().sum().values.tolist()
    isunique_list = df.nunique().values.tolist()
    dtypes_list = df.dtypes.tolist()

    list_stat_val = list(zip(columns_list, isnull_list, isunique_list, dtypes_list))
    df_stat_val = pd.DataFrame(
        list_stat_val, columns=["Name", "Null", "Unique", "Dtypes"]
    )
    print(tabulate(df_stat_val, headers="keys", tablefmt="psql"))
    return df.head()

In [ ]:
df_parquet = pd.read_parquet('/media/data1/muse_ge/ECG_ad202207_1453937_cat_labels_MUSE_vs_CARDIOLOGIST_v1.2.parquet')

In [ ]:
filtered_df = df_parquet[df_parquet['xml_path'].str.contains('MUSE_20221212_155611_05000.xml')]
display(filtered_df.head(n=5))


In [ ]:
import os
import sys

# Add necessary directories to sys.path
dir2 = os.path.abspath("/volume/DeepECG_Preprocess")
dir1 = os.path.dirname(dir2)
if dir1 not in sys.path:
    sys.path.append(dir1)

cli_xml2df_dir = os.path.abspath("/volume/DeepECG_Preprocess/utils")
if cli_xml2df_dir not in sys.path:
    sys.path.append(cli_xml2df_dir)

from DeepECG_Preprocess.utils.ecg_plotter import DataFrameECGPlotter

# Set matplotlib background to white
import matplotlib as mpl
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['axes.facecolor'] = 'white'

# Create a DataFrameECGPlotter instance
plotter = DataFrameECGPlotter(
    parquet_df=df_parquet,
    index=2,  # Using index 100 as specified
    npy_path_column="npy_path",
    patient_id_column="new_PatientID",
    diagnosis_column="diagnosis",
    dataset="MHI",
    out_dir="images",
    width=1000
)

# Plot the ECG
img, save_path = plotter.plot_ecg(save=False, anonymize=False, show_diagnosis=True)

# Display the image
display(img)

### MIMIC IV

In [ ]:
import pandas as pd 
mimic = pd.read_csv('/media/data1/ravram/MIMIC-IV/joined_annotation_file.csv')

# Identify rows containing 'hypertrophy' in the 'report' column
hypertrophy_rows = mimic[mimic['report'].str.contains('left ventricular hypertrophy', case=False, na=False)]

In [ ]:
import os
import sys

# Add necessary directories to sys.path
dir2 = os.path.abspath("/volume/DeepECG_Preprocess")
dir1 = os.path.dirname(dir2)
if dir1 not in sys.path:
    sys.path.append(dir1)

cli_xml2df_dir = os.path.abspath("/volume/DeepECG_Preprocess/utils")
if cli_xml2df_dir not in sys.path:
    sys.path.append(cli_xml2df_dir)

from DeepECG_Preprocess.utils.ecg_plotter import DataFrameECGPlotter

# Set matplotlib background to white
import matplotlib as mpl
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['axes.facecolor'] = 'white'

# Create a DataFrameECGPlotter instance
plotter = DataFrameECGPlotter(
    parquet_df=mimic,
    index=2736,
    npy_path_column="waveform_path",
    patient_id_column="new_PatientID",
    diagnosis_column="report",
    dataset="MIMICIV",
    out_dir=".",
    width=1000
)

# Plot the ECG
img, save_path = plotter.plot_ecg(save=False, anonymize=True, show_diagnosis=False)

# Display the image
display(img)

In [ ]:
## Example to plot using a path to NUMPY

from utils.multiprocessing_utils import process_multiple_ecgs
from utils.ecg_plotter import create_ecg_plotter


plotter = create_ecg_plotter("npy", mimic['waveform_path'].iloc[2736], dataset="MIMICIV")
img, save_path = plotter.plot_ecg(save=False, anonymize=True, title=mimic['report'].iloc[2736])

# Display the image
display(img) 